В этом задании будет рассмотрен открытый датасет с SMS-сообщениями, размеченных на спам ("spam") и не спам ("ham").

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

In [2]:
spam = pd.read_csv('SMSSpamCollection.txt', delimiter='\t', names=['label', 'text'])
spam.label.replace({'spam': 1, 'ham': 0}, inplace=True)
spam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   5572 non-null   int64 
 1   text    5572 non-null   object
dtypes: int64(1), object(1)
memory usage: 87.2+ KB


In [3]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(spam.text)
y = spam.label

In [4]:
lr = LogisticRegression()
cv = cross_val_score(lr, X, y, scoring='f1', cv=10)

In [5]:
mean_cv = np.mean(cv)
mean_cv

0.9311542822856882

In [6]:
with open("c5w3_5.txt", "w+") as file:
    file.write(str(round(mean_cv, 1)))

А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

"FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

"FreeMsg: Txt: claim your reward of 3 hours talk time"

"Have you visited the last lecture on physics?"

"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

"Only 99$"

Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

In [7]:
messages= vectorizer.transform([
    "FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
    "FreeMsg: Txt: claim your reward of 3 hours talk time",
    "Have you visited the last lecture on physics?",
    "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
    "Only 99$"
    ])

lr = LogisticRegression()
lr.fit(X, y)

predictions = lr.predict(messages).astype(str)
predictions

array(['1', '1', '0', '0', '0'], dtype='<U21')

In [8]:
with open("c5w3_6.txt", "w+") as file:
    file.write(" ".join(predictions))

Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [9]:
def run_experinent(estimator, ngram_ranges = [(2,2), (3,3), (1,3)]):
    avg_cv_scores = []
    for ngram_range in ngram_ranges:
        vectorizer=CountVectorizer(ngram_range=ngram_range)
        X = vectorizer.fit_transform(spam.text)
        lr = LogisticRegression()
        cv = cross_val_score(estimator, X, y, scoring='f1', cv=10)
        avg_cv_scores.append(str(round(np.mean(cv), 2)))
    return avg_cv_scores

In [10]:
results = run_experinent(LogisticRegression())
results

['0.82', '0.72', '0.92']

In [11]:
with open("c5w3_7.txt", "w+") as file:
    file.write(" ".join(results))

8. Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

In [12]:
results = run_experinent(MultinomialNB())
results

['0.65', '0.38', '0.89']

In [13]:
with open("c5w3_8.txt", "w+") as file:
    file.write(" ".join(results))

 Попробуйте использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [14]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(spam.text)
lr = LogisticRegression()
cv = cross_val_score(lr, X, y, scoring='f1', cv=10)
mean_cv_tfid = np.mean(cv)
f'mean_cv_tfid={mean_cv_tfid}; mean_cv_count={mean_cv}'

'mean_cv_tfid=0.8520118540708351; mean_cv_count=0.9311542822856882'

Видно, что качество упало.

In [15]:
with open("c5w3_9.txt", "w+") as file:
    file.write("-1")